# AWS

- hide: false
- toc: true
- comments: true
- categories: [python, pandas, aws]

Frequently used interaction patterns with AWS.

In [2]:
import pandas as pd

# Setup

- There are multiple ways to access your AWS account. I store config and credential files in `~/.aws` as discussed [here](https://docs.aws.amazon.com/cli/latest/userguide/cli-configure-files.html). AWS access methods find these files automatically so I don't have to worry about that.

- What I do have to worry about is choosing the appropriate profile depending on what AWS account I want to interact with (e.g. my personal one or one for work). This is different for each library, so I cover this below.

# `s3fs`

- Built by people at Dask, [`s3fs`](https://github.com/dask/s3fs) is built on top of `botocore` and provides a convenient way to interact with S3.

- While it can be used to read and write data, I find there are easier ways to do that and so I don't use the library for that.

In [2]:
import s3fs

## Navigate buckets

In [22]:
# establish connection
fs = s3fs.S3FileSystem()

# count items in s3 root bucket
len(fs.ls('/'))

2

## Choose profile

To choose a profile other than `default`, use:

In [18]:
# connect using a different profile
fs = s3fs.S3FileSystem(profile='tracker-fgu')
len(fs.ls('/'))

5

# Read and write directly from `Pandas`

- Pandas can read and write files to and from S3 directly if you provide the file name as `s3://<bucket>/<filename>`.

- By default, `Pandas` uses the default profile to access S3. Recent versions of `Pandas` have a `storage_options` parameter that can be used to provide, among other things, a profile name.

## Basics

In [43]:
# read using default profile 

fp = 's3://fgu-samples/transactions.parquet'
df = pd.read_parquet(fp)
df.shape

(1168943, 21)

In [5]:
# read using custom profile

fp = 's3://temp-mdb/data_XX7.parquet'
df = pd.read_parquet(fp, storage_options = dict(profile='tracker-fgu'))
df.shape

(9388334, 23)

This works well for simple jobs, but in a large project, passing the profile information to each read and write call is cumbersome and ugly.

## Simple improvement using `functools.partial` 

`functools.partial`provides a simple solution, as it allows me to create a custom function with a fixed storage options argument.

In [22]:
import functools

options = dict(storage_options=dict(profile='tracker-fgu'))
read_parquet_s3 = functools.partial(pd.read_parquet, **options)
df = read_parquet_s3(fp)
df.shape

(9388334, 23)

## More flexible solution with custom function

Often, I run projects on my Mac for testing and a virtual machine to run the full code. In this case, I need a way to automatically provide the correct profile name.

In [18]:
import platform

In [21]:
platform.node()

'FabsMacBook.local'

In [ ]:
def make_s3_funcs():
    """
    Return readers and writers with frozen AWS profile name.
    """
    # identify required profile (depends on project)
    if platform.node() == 'FabsMacBook.local':
        profile = 'tracker-fgu'
    else:
        profile = 'default'
        
    # create partial readers and writers
    options = dict(storage_options=dict(profile=profile))
    read_csv_s3 = functools.partial(pd.read_csv, **options)
    write_csv_s3 = functools.partial(pd.write_csv, **options)
    read_parquet_s3 = functools.partial(pd.read_parquet, **options)
    write_parquet_s3 = functools.partial(pd.write_parquet, **options)
    
    
    
    

The above is not ideal, as it requires cumbersome unpacking of return. Maybe using decorator is better.

# `awswrangler`

A new [library](https://github.com/awslabs/aws-data-wrangler) from AWS labs for Pandas interaction with a number of AWS services. Looks very promising, but haven't had any use for it thus far.